In [1]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [2]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC // svm = SVC(probability=True)

D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

In [3]:
def runAttack(model_wrapper, n, path, target_ds):
    attack = textattack.attack_recipes.TextFoolerJin2019.build(model_wrapper)
    attack_args = textattack.AttackArgs(num_examples=n, log_to_csv=path, shuffle=True, #parallel=True,
                                        checkpoint_interval=5, checkpoint_dir='checkpoints', 
                                        disable_stdout=True)
    attacker = textattack.Attacker(attack, target_ds, attack_args)
    attacker.attack_dataset()

In [4]:
# classifiers
def adaptation(train_set, test_set):
    lr = LogisticRegression(solver='saga', multi_class='multinomial')

    rf = RandomForestClassifier(max_depth=7)
    knn = KNeighborsClassifier(n_neighbors=7)
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]
    
    for key, clf in {'Log Reg:':lr, 'Random Forest:':rf, 'KNN':knn}.items():
        clf.fit(trainX, trainY) # fit train data
        pred = clf.predict(testX) # predict test set

        # print accuracy
        print(key, (pred==np.array(testY)).sum()/len(pred))

    return lr, rf, knn

In [5]:
target_ds_mr = textattack.datasets.HuggingFaceDataset('rotten_tomatoes', split='test')

textattack: Loading datasets dataset rotten_tomatoes, split test.


### BERT-based-uncased

In [6]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [7]:
bertmrtrain = pd.read_csv('dataset_embeddings/bertmrtrain.txt', header=None)
bertmrtest = pd.read_csv('dataset_embeddings/bertmrtest.txt', header=None)
lr_bert_mr, rf_bert_mr, knn_bert_mr = adaptation(bertmrtrain, bertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg: 0.8114446529080676
Random Forest: 0.7317073170731707
KNN 0.6801125703564728


In [8]:
# custom model wrapper for CLIP
class BERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [9]:
bert_model_wrapper_lr_mr = BERTModelWrapper(bert_tokenizer, bert_model, lr_bert_mr)
bert_model_wrapper_rf_mr = BERTModelWrapper(bert_tokenizer, bert_model, rf_bert_mr)
bert_model_wrapper_knn_mr = BERTModelWrapper(bert_tokenizer, bert_model, knn_bert_mr)

In [10]:
runAttack(bert_model_wrapper_lr_mr, -1, 'attack_results_new/bertmrattacklr.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacklr.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [00:36<2:09:05,  7.30s/it]textattack: Saving checkpoint under "checkpoints\1719930531624.ta.chkpt" at 2024-07-02 22:28:51 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:   1%|▏                        | 6/1066 [00:36<1:47:56,  6.11s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                      | 10/1066 [00:46<1:21:16,  4.62s/it]textattack: Saving checkpoint under "checkpoints\1719930541302.ta.chkpt" at 2024-07-02 22:29:01 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   1%|▎                     | 15/1066 [01:14<1:27:07,  4.97s/it]textattack: Saving checkpoint under "checkpoints\1719930569729.ta.chkpt" at 2024-07-02 22:29:29 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 842 / 15 / 198 / 1055:  99%|███████████████▊| 1055/1066 [49:03<00:30,  2.79s/it]textattack: Saving checkpoint under "checkpoints\1719933438959.ta.chkpt" at 2024-07-02 23:17:18 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 842 / 15 / 199 / 1056:  99%|███████████████▊| 1056/1066 [49:03<00:27,  2.79s/it]

[Succeeded / Failed / Skipped / Total] 846 / 15 / 199 / 1060:  99%|███████████████▉| 1060/1066 [49:20<00:16,  2.79s/it]textattack: Saving checkpoint under "checkpoints\1719933455523.ta.chkpt" at 2024-07-02 23:17:35 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 849 / 15 / 201 / 1065: 100%|███████████████▉| 1065/1066 [49:39<00:02,  2.80s/it]textattack: Saving checkpoint under "checkpoints\1719933474546.ta.chkpt" at 2024-07-02 23:17:54 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 850 / 15 / 201 / 1066: 100%|████████████████| 1066/1066 [49:41<00:00,  2.80s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 850    |
| Number of failed attacks:     | 15     |
| Number of skipped attacks:    | 201    |
| Original accuracy:            | 81.14% |
| Accuracy under attack:        | 1.41%  |
| Attack success rate:          | 98.27% |
| Average perturbed word %:     | 15.3%  |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 78.2   |
+-------------------------------+--------+


In [11]:
runAttack(bert_model_wrapper_rf_mr, -1, 'attack_results_new/bertmrattackrf.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackrf.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [00:18<1:05:26,  3.70s/it]textattack: Saving checkpoint under "checkpoints\1719933495125.ta.chkpt" at 2024-07-02 23:18:15 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▏                          | 7/1066 [00:18<47:02,  2.67s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                        | 10/1066 [00:33<58:30,  3.32s/it]textattack: Saving checkpoint under "checkpoints\1719933509867.ta.chkpt" at 2024-07-02 23:18:29 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 0 / 5 / 15:   1%|▎                       | 15/1066 [00:45<53:17,  3.04s/it]textattack: Saving checkpoint under "checkpoints\1719933522259.ta.chkpt" at 2024-07-02 23:18:42 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 739 / 31 / 285 / 1055:  99%|███████████████▊| 1055/1066 [45:43<00:28,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1719936220379.ta.chkpt" at 2024-07-03 00:03:40 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 744 / 31 / 285 / 1060:  99%|███████████████▉| 1060/1066 [46:02<00:15,  2.61s/it]textattack: Saving checkpoint under "checkpoints\1719936239500.ta.chkpt" at 2024-07-03 00:03:59 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 748 / 31 / 286 / 1065: 100%|███████████████▉| 1065/1066 [46:10<00:02,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1719936246788.ta.chkpt" at 2024-07-03 00:04:06 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 749 / 31 / 286 / 1066: 100%|████████████████| 1066/1066 [46:10<00:00,  2.60s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 749    |
| Number of failed attacks:     | 31     |
| Number of skipped attacks:    | 286    |
| Original accuracy:            | 73.17% |
| Accuracy under attack:        | 2.91%  |
| Attack success rate:          | 96.03% |
| Average perturbed word %:     | 13.63% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 77.51  |
+-------------------------------+--------+


In [12]:
runAttack(bert_model_wrapper_knn_mr, -1, 'attack_results_new/bertmrattackknn.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackknn.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 0 / 2 / 5:   0%|                         | 5/1066 [00:18<1:05:25,  3.70s/it]textattack: Saving checkpoint under "checkpoints\1719936266043.ta.chkpt" at 2024-07-03 00:04:26 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:   1%|▏                          | 6/1066 [00:18<54:39,  3.09s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                        | 10/1066 [00:24<42:22,  2.41s/it]textattack: Saving checkpoint under "checkpoints\1719936271622.ta.chkpt" at 2024-07-03 00:04:31 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 1 / 6 / 15:   1%|▎                      | 15/1066 [01:11<1:22:57,  4.74s/it]textattack: Saving checkpoint under "checkpoints\1719936318587.ta.chkpt" at 2024-07-03 00:05:18 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 709 / 9 / 337 / 1055:  99%|████████████████▊| 1055/1066 [28:01<00:17,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1719937928891.ta.chkpt" at 2024-07-03 00:32:08 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 713 / 9 / 338 / 1060:  99%|████████████████▉| 1060/1066 [28:11<00:09,  1.60s/it]textattack: Saving checkpoint under "checkpoints\1719937938865.ta.chkpt" at 2024-07-03 00:32:18 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 716 / 9 / 340 / 1065: 100%|████████████████▉| 1065/1066 [28:17<00:01,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1719937944960.ta.chkpt" at 2024-07-03 00:32:24 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 716 / 9 / 341 / 1066: 100%|█████████████████| 1066/1066 [28:17<00:00,  1.59s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 716    |
| Number of failed attacks:     | 9      |
| Number of skipped attacks:    | 341    |
| Original accuracy:            | 68.01% |
| Accuracy under attack:        | 0.84%  |
| Attack success rate:          | 98.76% |
| Average perturbed word %:     | 8.92%  |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 51.48  |
+-------------------------------+--------+


### DistilBERT-based-uncased

In [13]:
# bert-based-uncased
model_path2 = 'distilbert/distilbert-base-uncased'

disbert_model = transformers.AutoModel.from_pretrained(model_path2)
disbert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path2)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
disbertmrtrain = pd.read_csv('dataset_embeddings/disbertmrtrain.txt', header=None)
disbertmrtest = pd.read_csv('dataset_embeddings/disbertmrtest.txt', header=None)
lr_disbert_mr, rf_disbert_mr, knn_disbert_mr = adaptation(disbertmrtrain, disbertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg: 0.7945590994371482
Random Forest: 0.7682926829268293
KNN 0.7298311444652908


In [15]:
# custom model wrapper for DistilBERT
class DisBERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).last_hidden_state.mean(axis=1)
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [16]:
disbert_model_wrapper_lr_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, lr_disbert_mr)
disbert_model_wrapper_rf_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, rf_disbert_mr)
disbert_model_wrapper_knn_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, knn_disbert_mr)

In [17]:
runAttack(disbert_model_wrapper_lr_mr, -1, 'attack_results_new/disbertmrattacklr.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacklr.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|▏                          | 5/1066 [00:16<58:25,  3.30s/it]textattack: Saving checkpoint under "checkpoints\1719938004280.ta.chkpt" at 2024-07-03 00:33:24 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▏                          | 7/1066 [00:16<41:50,  2.37s/it]

[Succeeded / Failed / Skipped / Total] 7 / 0 / 3 / 10:   1%|▏                        | 10/1066 [00:21<37:10,  2.11s/it]textattack: Saving checkpoint under "checkpoints\1719938008878.ta.chkpt" at 2024-07-03 00:33:28 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   1%|▎                       | 15/1066 [00:35<41:45,  2.38s/it]textattack: Saving checkpoint under "checkpoints\1719938023516.ta.chkpt" at 2024-07-03 00:33:43 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 826 / 12 / 217 / 1055:  99%|███████████████▊| 1055/1066 [26:40<00:16,  1.52s/it]textattack: Saving checkpoint under "checkpoints\1719939588558.ta.chkpt" at 2024-07-03 00:59:48 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 826 / 12 / 218 / 1056:  99%|███████████████▊| 1056/1066 [26:40<00:15,  1.52s/it]

[Succeeded / Failed / Skipped / Total] 830 / 12 / 218 / 1060:  99%|███████████████▉| 1060/1066 [26:51<00:09,  1.52s/it]textattack: Saving checkpoint under "checkpoints\1719939599341.ta.chkpt" at 2024-07-03 00:59:59 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 834 / 12 / 219 / 1065: 100%|███████████████▉| 1065/1066 [27:02<00:01,  1.52s/it]textattack: Saving checkpoint under "checkpoints\1719939609858.ta.chkpt" at 2024-07-03 01:00:09 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 835 / 12 / 219 / 1066: 100%|████████████████| 1066/1066 [27:03<00:00,  1.52s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 835    |
| Number of failed attacks:     | 12     |
| Number of skipped attacks:    | 219    |
| Original accuracy:            | 79.46% |
| Accuracy under attack:        | 1.13%  |
| Attack success rate:          | 98.58% |
| Average perturbed word %:     | 15.69% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 76.94  |
+-------------------------------+--------+


In [18]:
runAttack(disbert_model_wrapper_rf_mr, -1, 'attack_results_new/disbertmrattackrf.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackrf.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [00:17<1:03:15,  3.58s/it]textattack: Saving checkpoint under "checkpoints\1719939628743.ta.chkpt" at 2024-07-03 01:00:28 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:   1%|▏                          | 6/1066 [00:18<53:01,  3.00s/it]

[Succeeded / Failed / Skipped / Total] 8 / 0 / 2 / 10:   1%|▏                        | 10/1066 [00:28<49:44,  2.83s/it]textattack: Saving checkpoint under "checkpoints\1719939639120.ta.chkpt" at 2024-07-03 01:00:39 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 1 / 3 / 15:   1%|▎                       | 15/1066 [00:48<56:39,  3.23s/it]textattack: Saving checkpoint under "checkpoints\1719939659375.ta.chkpt" at 2024-07-03 01:00:59 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 761 / 48 / 246 / 1055:  99%|███████████████▊| 1055/1066 [32:06<00:20,  1.83s/it]textattack: Saving checkpoint under "checkpoints\1719941537434.ta.chkpt" at 2024-07-03 01:32:17 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 765 / 49 / 246 / 1060:  99%|███████████████▉| 1060/1066 [32:21<00:10,  1.83s/it]textattack: Saving checkpoint under "checkpoints\1719941552352.ta.chkpt" at 2024-07-03 01:32:32 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 768 / 50 / 247 / 1065: 100%|███████████████▉| 1065/1066 [32:36<00:01,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1719941567589.ta.chkpt" at 2024-07-03 01:32:47 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 769 / 50 / 247 / 1066: 100%|████████████████| 1066/1066 [32:37<00:00,  1.84s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 769    |
| Number of failed attacks:     | 50     |
| Number of skipped attacks:    | 247    |
| Original accuracy:            | 76.83% |
| Accuracy under attack:        | 4.69%  |
| Attack success rate:          | 93.89% |
| Average perturbed word %:     | 16.57% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 91.7   |
+-------------------------------+--------+


In [19]:
runAttack(disbert_model_wrapper_knn_mr, -1, 'attack_results_new/disbertmrattackknn.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackknn.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   0%|                         | 5/1066 [00:17<1:02:45,  3.55s/it]textattack: Saving checkpoint under "checkpoints\1719941586476.ta.chkpt" at 2024-07-03 01:33:06 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 2 / 7:   1%|▏                          | 7/1066 [00:17<45:01,  2.55s/it]

[Succeeded / Failed / Skipped / Total] 7 / 1 / 2 / 10:   1%|▏                        | 10/1066 [00:28<50:42,  2.88s/it]textattack: Saving checkpoint under "checkpoints\1719941597540.ta.chkpt" at 2024-07-03 01:33:17 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 1 / 3 / 15:   1%|▎                       | 15/1066 [00:38<45:00,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1719941607276.ta.chkpt" at 2024-07-03 01:33:27 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 706 / 64 / 285 / 1055:  99%|███████████████▊| 1055/1066 [28:53<00:18,  1.64s/it]textattack: Saving checkpoint under "checkpoints\1719943301988.ta.chkpt" at 2024-07-03 02:01:41 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 709 / 65 / 286 / 1060:  99%|███████████████▉| 1060/1066 [29:04<00:09,  1.65s/it]textattack: Saving checkpoint under "checkpoints\1719943313616.ta.chkpt" at 2024-07-03 02:01:53 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 712 / 65 / 288 / 1065: 100%|███████████████▉| 1065/1066 [29:12<00:01,  1.65s/it]textattack: Saving checkpoint under "checkpoints\1719943320854.ta.chkpt" at 2024-07-03 02:02:00 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 713 / 65 / 288 / 1066: 100%|████████████████| 1066/1066 [29:12<00:00,  1.64s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 713    |
| Number of failed attacks:     | 65     |
| Number of skipped attacks:    | 288    |
| Original accuracy:            | 72.98% |
| Accuracy under attack:        | 6.1%   |
| Attack success rate:          | 91.65% |
| Average perturbed word %:     | 10.88% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 82.68  |
+-------------------------------+--------+


### CLIP

In [20]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
clipmrtrain = pd.read_csv('dataset_embeddings/clipmrtrain.txt', header=None)
clipmrtest = pd.read_csv('dataset_embeddings/clipmrtest.txt', header=None)
lr_clip_mr, rf_clip_mr, knn_clip_mr = adaptation(clipmrtrain, clipmrtest)

Log Reg: 0.7495309568480301
Random Forest: 0.7054409005628518
KNN 0.6885553470919324


In [22]:
# custom model wrapper for CLIP
class CLIPModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [23]:
clip_model_wrapper_lr_mr = CLIPModelWrapper(clip_processor, text_encoder, lr_clip_mr)
clip_model_wrapper_rf_mr = CLIPModelWrapper(clip_processor, text_encoder, rf_clip_mr)
clip_model_wrapper_knn_mr = CLIPModelWrapper(clip_processor, text_encoder, knn_clip_mr)

In [24]:
runAttack(clip_model_wrapper_lr_mr, -1, 'attack_results_new/clipmrattacklr.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacklr.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   0%|                         | 5/1066 [00:19<1:07:14,  3.80s/it]textattack: Saving checkpoint under "checkpoints\1719943371712.ta.chkpt" at 2024-07-03 02:02:51 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:   1%|▏                          | 6/1066 [00:19<56:06,  3.18s/it]

[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10:   1%|▏                        | 10/1066 [00:26<45:46,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1719943378713.ta.chkpt" at 2024-07-03 02:02:58 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 0 / 1 / 15:   1%|▎                       | 15/1066 [00:43<50:55,  2.91s/it]textattack: Saving checkpoint under "checkpoints\1719943396309.ta.chkpt" at 2024-07-03 02:03:16 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 787 / 3 / 265 / 1055:  99%|████████████████▊| 1055/1066 [23:26<00:14,  1.33s/it]textattack: Saving checkpoint under "checkpoints\1719944758893.ta.chkpt" at 2024-07-03 02:25:58 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 787 / 3 / 266 / 1056:  99%|████████████████▊| 1056/1066 [23:26<00:13,  1.33s/it]

[Succeeded / Failed / Skipped / Total] 791 / 3 / 266 / 1060:  99%|████████████████▉| 1060/1066 [23:31<00:07,  1.33s/it]textattack: Saving checkpoint under "checkpoints\1719944764144.ta.chkpt" at 2024-07-03 02:26:04 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 795 / 3 / 267 / 1065: 100%|████████████████▉| 1065/1066 [23:41<00:01,  1.33s/it]textattack: Saving checkpoint under "checkpoints\1719944773802.ta.chkpt" at 2024-07-03 02:26:13 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 796 / 3 / 267 / 1066: 100%|█████████████████| 1066/1066 [23:41<00:00,  1.33s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 796    |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 267    |
| Original accuracy:            | 74.95% |
| Accuracy under attack:        | 0.28%  |
| Attack success rate:          | 99.62% |
| Average perturbed word %:     | 13.37% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 64.43  |
+-------------------------------+--------+


In [25]:
runAttack(clip_model_wrapper_rf_mr, -1, 'attack_results_new/clipmrattackrf.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackrf.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   0%|                         | 5/1066 [00:18<1:06:56,  3.79s/it]textattack: Saving checkpoint under "checkpoints\1719944793462.ta.chkpt" at 2024-07-03 02:26:33 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 2 / 7:   1%|▏                          | 7/1066 [00:19<47:59,  2.72s/it]

[Succeeded / Failed / Skipped / Total] 8 / 0 / 2 / 10:   1%|▏                        | 10/1066 [00:29<51:27,  2.92s/it]textattack: Saving checkpoint under "checkpoints\1719944803776.ta.chkpt" at 2024-07-03 02:26:43 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 0 / 3 / 15:   1%|▎                       | 15/1066 [00:41<48:09,  2.75s/it]textattack: Saving checkpoint under "checkpoints\1719944815778.ta.chkpt" at 2024-07-03 02:26:55 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 707 / 36 / 312 / 1055:  99%|███████████████▊| 1055/1066 [27:37<00:17,  1.57s/it]textattack: Saving checkpoint under "checkpoints\1719946432273.ta.chkpt" at 2024-07-03 02:53:52 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 707 / 36 / 313 / 1056:  99%|███████████████▊| 1056/1066 [27:37<00:15,  1.57s/it]

[Succeeded / Failed / Skipped / Total] 711 / 36 / 313 / 1060:  99%|███████████████▉| 1060/1066 [27:42<00:09,  1.57s/it]textattack: Saving checkpoint under "checkpoints\1719946437492.ta.chkpt" at 2024-07-03 02:53:57 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 715 / 36 / 314 / 1065: 100%|███████████████▉| 1065/1066 [27:51<00:01,  1.57s/it]textattack: Saving checkpoint under "checkpoints\1719946446510.ta.chkpt" at 2024-07-03 02:54:06 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 716 / 36 / 314 / 1066: 100%|████████████████| 1066/1066 [27:52<00:00,  1.57s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 716    |
| Number of failed attacks:     | 36     |
| Number of skipped attacks:    | 314    |
| Original accuracy:            | 70.54% |
| Accuracy under attack:        | 3.38%  |
| Attack success rate:          | 95.21% |
| Average perturbed word %:     | 13.41% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 76.72  |
+-------------------------------+--------+


In [26]:
runAttack(clip_model_wrapper_knn_mr, -1, 'attack_results_new/clipmrattackknn.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackknn.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:   0%|▏                          | 5/1066 [00:15<55:43,  3.15s/it]textattack: Saving checkpoint under "checkpoints\1719946462764.ta.chkpt" at 2024-07-03 02:54:22 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 1 / 2 / 10:   1%|▏                        | 10/1066 [00:19<34:46,  1.98s/it]textattack: Saving checkpoint under "checkpoints\1719946466766.ta.chkpt" at 2024-07-03 02:54:26 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 1 / 3 / 15:   1%|▎                       | 15/1066 [00:31<36:58,  2.11s/it]textattack: Saving checkpoint under "checkpoints\1719946478674.ta.chkpt" at 2024-07-03 02:54:38 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 661 / 63 / 331 / 1055:  99%|███████████████▊| 1055/1066 [29:45<00:18,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1719948232795.ta.chkpt" at 2024-07-03 03:23:52 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 666 / 63 / 331 / 1060:  99%|███████████████▉| 1060/1066 [29:51<00:10,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1719948238473.ta.chkpt" at 2024-07-03 03:23:58 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 670 / 63 / 332 / 1065: 100%|███████████████▉| 1065/1066 [29:57<00:01,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1719948244114.ta.chkpt" at 2024-07-03 03:24:04 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 671 / 63 / 332 / 1066: 100%|████████████████| 1066/1066 [29:57<00:00,  1.69s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 671    |
| Number of failed attacks:     | 63     |
| Number of skipped attacks:    | 332    |
| Original accuracy:            | 68.86% |
| Accuracy under attack:        | 5.91%  |
| Attack success rate:          | 91.42% |
| Average perturbed word %:     | 10.07% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 79.69  |
+-------------------------------+--------+


#### Add SVC

In [33]:
# classifiers
from sklearn.svm import SVC

def adaptation2(train_set, test_set):
    svc = SVC(kernel='linear', probability=True)
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]
    
    svc.fit(trainX, trainY) # fit train data
    pred = svc.predict(testX) # predict test set

    # print accuracy
    print('SVC', (pred==np.array(testY)).sum()/len(pred))

    return svc

In [34]:
svc_bert = adaptation2(bertmrtrain, bertmrtest)
svc_disbert = adaptation2(disbertmrtrain, disbertmrtest)
svc_clip = adaptation2(clipmrtrain, clipmrtest)

SVC 0.8030018761726079
SVC 0.7851782363977486
SVC 0.7551594746716698


In [35]:
bert_model_wrapper_svc_mr = BERTModelWrapper(bert_tokenizer, bert_model, svc_bert)
disbert_model_wrapper_svc_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, svc_disbert)
clip_model_wrapper_svc_mr = CLIPModelWrapper(clip_processor, text_encoder, svc_clip)

In [36]:
runAttack(bert_model_wrapper_svc_mr, -1, 'attack_results_new/bertmrattacksvc.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacksvc.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       


  0%|                                                                              | 1/1066 [01:00<17:54:00, 60.51s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|                        | 1/1066 [01:00<18:02:38, 60.99s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|                         | 2/1066 [01:05<9:38:14, 32.61s/it]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   0%|                         | 2/1066 [01:05<9:38:38, 32.63s/it]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   0%|                         | 3/1066 [01:05<6:26:17, 21.80s/it]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   0%|                         | 3/1066 [01:05<6:26:22, 21.81s/it]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   0%|                         | 4/1066 [01:10<5:12:41, 17.67s/it]
[Succeeded / Failed / Skipped / Total] 3 / 0 / 1 / 4:   0%|                         | 4/1066 [01:10<5:12:48, 17.67s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▏                        | 6/1066 [01:18<3:51:06, 13.08s/it]
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:   1%|▏                        | 6/1066 [01:18<3:51:12, 13.09s/it]
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▏                        | 7/1066 [01:18<3:18:10, 11.23s/it]
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▏                        | 8/1066 [01:24<3:05:27, 10.52s/it]
[Succeeded / Failed / Skipped / Total] 5 / 0 / 3 / 8:   1%|▏                        | 8/1066 [01:24<3:05:32, 10.52s/it]
[Succeeded / Failed / Skipped / Total] 6 / 0 / 3 / 9:   1%|▏                        | 9/1066 [01:30<2:57:18, 10.06s/it]
[Succeeded / Failed / Skipped / Total] 6 / 0 / 3 / 9:   1%|▏                       | 10/1066 [01:30<2:39:38,  9.07s/it]
[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                      | 10/1066 [01:30<2:39:41,  9.07s/it]textattack: Saving checkpoint under "che


[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                      | 10/1066 [01:43<3:01:53, 10.33s/it]
[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                      | 11/1066 [01:57<3:07:43, 10.68s/it]
[Succeeded / Failed / Skipped / Total] 7 / 0 / 4 / 11:   1%|▏                      | 11/1066 [01:57<3:07:45, 10.68s/it]
[Succeeded / Failed / Skipped / Total] 7 / 0 / 4 / 11:   1%|▎                      | 12/1066 [01:59<2:54:36,  9.94s/it]
[Succeeded / Failed / Skipped / Total] 8 / 0 / 4 / 12:   1%|▎                      | 12/1066 [01:59<2:54:37,  9.94s/it]
[Succeeded / Failed / Skipped / Total] 8 / 0 / 4 / 12:   1%|▎                      | 13/1066 [02:02<2:45:58,  9.46s/it]
[Succeeded / Failed / Skipped / Total] 9 / 0 / 4 / 13:   1%|▎                      | 13/1066 [02:02<2:46:00,  9.46s/it]
[Succeeded / Failed / Skipped / Total] 9 / 0 / 4 / 13:   1%|▎                      | 14/1066 [02:05<2:36:42,  8.94s/it]
[Succeeded / Failed / Skipped / Total] 

...


[Succeeded / Failed / Skipped / Total] 828 / 17 / 206 / 1051:  99%|███████████████▊| 1051/1066 [57:22<00:49,  3.28s/it]
[Succeeded / Failed / Skipped / Total] 828 / 17 / 206 / 1051:  99%|███████████████▊| 1052/1066 [57:24<00:45,  3.27s/it]
[Succeeded / Failed / Skipped / Total] 829 / 17 / 206 / 1052:  99%|███████████████▊| 1052/1066 [57:24<00:45,  3.27s/it]
[Succeeded / Failed / Skipped / Total] 829 / 17 / 207 / 1053:  99%|███████████████▊| 1053/1066 [57:24<00:42,  3.27s/it]
[Succeeded / Failed / Skipped / Total] 829 / 17 / 207 / 1053:  99%|███████████████▊| 1054/1066 [57:26<00:39,  3.27s/it]
[Succeeded / Failed / Skipped / Total] 830 / 17 / 207 / 1054:  99%|███████████████▊| 1054/1066 [57:26<00:39,  3.27s/it]
[Succeeded / Failed / Skipped / Total] 831 / 17 / 207 / 1055:  99%|███████████████▊| 1055/1066 [57:26<00:35,  3.27s/it]textattack: Saving checkpoint under "checkpoints\1720009060905.ta.chkpt" at 2024-07-03 20:17:40 after 1055 attacks.



[Succeeded / Failed / Skipped / Total] 831 / 17 / 207 / 1055:  99%|███████████████▊| 1056/1066 [57:26<00:32,  3.26s/it]
[Succeeded / Failed / Skipped / Total] 831 / 17 / 208 / 1056:  99%|███████████████▊| 1056/1066 [57:27<00:32,  3.26s/it]
[Succeeded / Failed / Skipped / Total] 832 / 17 / 208 / 1057:  99%|███████████████▊| 1057/1066 [57:29<00:29,  3.26s/it]
[Succeeded / Failed / Skipped / Total] 832 / 17 / 208 / 1057:  99%|███████████████▉| 1058/1066 [57:33<00:26,  3.26s/it]
[Succeeded / Failed / Skipped / Total] 833 / 17 / 208 / 1058:  99%|███████████████▉| 1058/1066 [57:33<00:26,  3.26s/it]
[Succeeded / Failed / Skipped / Total] 834 / 17 / 208 / 1059:  99%|███████████████▉| 1059/1066 [57:41<00:22,  3.27s/it]
[Succeeded / Failed / Skipped / Total] 834 / 17 / 208 / 1059:  99%|███████████████▉| 1060/1066 [57:45<00:19,  3.27s/it]
[Succeeded / Failed / Skipped / Total] 835 / 17 / 208 / 1060:  99%|███████████████▉| 1060/1066 [57:45<00:19,  3.27s/it]textattack: Saving checkpoint under "chec


[Succeeded / Failed / Skipped / Total] 835 / 17 / 208 / 1060: 100%|███████████████▉| 1061/1066 [57:49<00:16,  3.27s/it]
[Succeeded / Failed / Skipped / Total] 836 / 17 / 208 / 1061: 100%|███████████████▉| 1061/1066 [57:49<00:16,  3.27s/it]
[Succeeded / Failed / Skipped / Total] 836 / 17 / 208 / 1061: 100%|███████████████▉| 1062/1066 [57:54<00:13,  3.27s/it]
[Succeeded / Failed / Skipped / Total] 837 / 17 / 208 / 1062: 100%|███████████████▉| 1062/1066 [57:54<00:13,  3.27s/it]
[Succeeded / Failed / Skipped / Total] 837 / 17 / 208 / 1062: 100%|███████████████▉| 1063/1066 [58:08<00:09,  3.28s/it]
[Succeeded / Failed / Skipped / Total] 838 / 17 / 208 / 1063: 100%|███████████████▉| 1063/1066 [58:08<00:09,  3.28s/it]
[Succeeded / Failed / Skipped / Total] 838 / 17 / 208 / 1063: 100%|███████████████▉| 1064/1066 [58:08<00:06,  3.28s/it]
[Succeeded / Failed / Skipped / Total] 838 / 17 / 209 / 1064: 100%|███████████████▉| 1064/1066 [58:08<00:06,  3.28s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 838 / 17 / 210 / 1065: 100%|████████████████| 1066/1066 [58:11<00:00,  3.28s/it]
[Succeeded / Failed / Skipped / Total] 839 / 17 / 210 / 1066: 100%|████████████████| 1066/1066 [58:11<00:00,  3.28s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 839    |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 210    |
| Original accuracy:            | 80.3%  |
| Accuracy under attack:        | 1.59%  |
| Attack success rate:          | 98.01% |
| Average perturbed word %:     | 15.16% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 77.43  |
+-------------------------------+--------+


In [37]:
runAttack(disbert_model_wrapper_svc_mr, -1, 'attack_results_new/disbertmrattacksvc.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacksvc.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [00:31<1:52:43,  6.37s/it]textattack: Saving checkpoint under "checkpoints\1720009137610.ta.chkpt" at 2024-07-03 20:18:57 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▏                        | 7/1066 [00:32<1:20:46,  4.58s/it]

[Succeeded / Failed / Skipped / Total] 7 / 0 / 3 / 10:   1%|▏                      | 10/1066 [00:39<1:09:55,  3.97s/it]textattack: Saving checkpoint under "checkpoints\1720009145453.ta.chkpt" at 2024-07-03 20:19:05 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   1%|▎                     | 15/1066 [00:56<1:06:21,  3.79s/it]textattack: Saving checkpoint under "checkpoints\1720009162540.ta.chkpt" at 2024-07-03 20:19:22 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 814 / 15 / 226 / 1055:  99%|███████████████▊| 1055/1066 [31:49<00:19,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1720011014724.ta.chkpt" at 2024-07-03 20:50:14 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 814 / 15 / 227 / 1056:  99%|███████████████▊| 1056/1066 [31:49<00:18,  1.81s/it]

[Succeeded / Failed / Skipped / Total] 818 / 15 / 227 / 1060:  99%|███████████████▉| 1060/1066 [31:59<00:10,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1720011025456.ta.chkpt" at 2024-07-03 20:50:25 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 822 / 15 / 228 / 1065: 100%|███████████████▉| 1065/1066 [32:11<00:01,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1720011036721.ta.chkpt" at 2024-07-03 20:50:36 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 823 / 15 / 228 / 1066: 100%|████████████████| 1066/1066 [32:12<00:00,  1.81s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 823    |
| Number of failed attacks:     | 15     |
| Number of skipped attacks:    | 228    |
| Original accuracy:            | 78.61% |
| Accuracy under attack:        | 1.41%  |
| Attack success rate:          | 98.21% |
| Average perturbed word %:     | 15.7%  |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 77.47  |
+-------------------------------+--------+


In [38]:
runAttack(clip_model_wrapper_svc_mr, -1, 'attack_results_new/clipmrattacksvc.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacksvc.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   0%|                         | 5/1066 [00:26<1:34:15,  5.33s/it]textattack: Saving checkpoint under "checkpoints\1720011064617.ta.chkpt" at 2024-07-03 20:51:04 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10:   1%|▏                      | 10/1066 [00:34<1:01:07,  3.47s/it]textattack: Saving checkpoint under "checkpoints\1720011072691.ta.chkpt" at 2024-07-03 20:51:12 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 0 / 2 / 15:   1%|▎                     | 15/1066 [00:53<1:02:12,  3.55s/it]textattack: Saving checkpoint under "checkpoints\1720011091232.ta.chkpt" at 2024-07-03 20:51:31 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 791 / 5 / 259 / 1055:  99%|████████████████▊| 1055/1066 [26:36<00:16,  1.51s/it]textattack: Saving checkpoint under "checkpoints\1720012634638.ta.chkpt" at 2024-07-03 21:17:14 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 795 / 5 / 260 / 1060:  99%|████████████████▉| 1060/1066 [26:43<00:09,  1.51s/it]textattack: Saving checkpoint under "checkpoints\1720012641225.ta.chkpt" at 2024-07-03 21:17:21 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 799 / 5 / 261 / 1065: 100%|████████████████▉| 1065/1066 [26:55<00:01,  1.52s/it]textattack: Saving checkpoint under "checkpoints\1720012653803.ta.chkpt" at 2024-07-03 21:17:33 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 800 / 5 / 261 / 1066: 100%|█████████████████| 1066/1066 [26:56<00:00,  1.52s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 800    |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 261    |
| Original accuracy:            | 75.52% |
| Accuracy under attack:        | 0.47%  |
| Attack success rate:          | 99.38% |
| Average perturbed word %:     | 13.15% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 63.64  |
+-------------------------------+--------+


In [8]:
import numpy as np 
1 - 0.6/np.pi

0.8090140682897256